In [ ]:
# WITH clauses: The main query then refers to the subquery by it's alias name.
    WITH [alias_name] AS ([subquery])
    SELECT [main_query]

In [ ]:
# show summary data for every playlist in the Chinook database:
#    Use a WITH clause to create a named subquery with the following info:
#        The unique ID for the playlist.
#        The name of the playlist.
#        The name of each track from the playlist.
#        The length of the each track in seconds.
#    Final table should have the following columns, in order:
#        playlist_id - the unique ID for the playlist.
#        playlist_name - The name of the playlist.
#        number_of_tracks - A count of the number of tracks in the playlist.
#        length_seconds - The sum of the length of the playlist in seconds.
#    The results should be sorted by playlist_id in ascending order.

    WITH playlist_info AS
            (SELECT p.playlist_id, 
                    p.name playlist_name, 
                    t.name track_name,
                    (t.milliseconds/1000) seconds
     FROM playlist p
     LEFT JOIN playlist_track pt ON pt.playlist_id = p.playlist_id
     LEFT JOIN track t ON t.track_id = pt.track_id
             )
    SELECT playlist_id, 
           playlist_name, 
           COUNT(track_name) number_of_tracks,
           SUM(seconds) length_seconds
    FROM playlist_info
    GROUP by 1, 2
    ORDER BY 1 ASC

In [ ]:
# VIEW can be used in all future queries
    CREATE VIEW chinook.customer_2 AS
            SELECT * FROM chinook.customer;
        
    DROP VIEW chinook.customer_2; # before can redefine a view

In [ ]:
    CREATE VIEW chinook.customer_usa AS 
            SELECT * FROM chinook.customer
            WHERE country = "USA";  

In [ ]:
# Create a view called customer_gt_90_dollars:
#    The view should contain the columns from customers, in their original order.
#    The view should contain only customers who have purchased more than $90 in tracks 
#         from the store.
#    Write a second query to display newly created view: 
#    Make sure use a semicolon (;) to indicate the end of each query    
    CREATE VIEW chinook.customer_gt_90_dollars AS
                SELECT c.* 
                FROM chinook.customer c
                INNER JOIN chinook.invoice i ON c.customer_id=i.customer_id
                GROUP BY 1 
                HAVING SUM(i.total) > 90;
    SELECT * 
    FROM chinook.customer_gt_90_dollars;

In [ ]:
# UNION: join rows from tables and/or views while joins are used to join columns.  
# The statements before and after UNION must have the same number of columns, 
#    with compatible types in order 

In [ ]:
# UNION: Selects rows that occur in either statement.
# INTERSECT: Selects rows that occur in both statements. 
# EXCEPT: Selects rows that occur in the first statement, 
#         but don't occur in the second statement. 

In [ ]:
# how many customers that are in the USA and have purchased more than $90 
# are assigned to each sales support agent. Assume no two employees have the same name.
# Expected result: in order:
#    employee_name - The first_name and last_name of the employee separated by a space
#    customers_usa_gt_90 - The number of customer assigned to that employee 
#       that are both from the USA and have have purchased more than $90 worth of tracks.
# The result should include all employees with the title "Sales Support Agent", 
#   but not employees with any other title.
     
    WITH customers_usa_gt AS
        (
         SELECT * FROM customer_usa
         INTERSECT 
         SELECT * FROM customer_gt_90_dollars
        ) 
    SELECT 
        e.first_name || " " || e.last_name employee_name,
        COUNT(c.customer_id) customers_usa_gt_90
    FROM employee e
    LEFT JOIN customers_usa_gt c ON c.support_rep_id = e.employee_id
    WHERE e.title = 'Sales Support Agent'
    GROUP BY employee_name
    ORDER BY employee_name

In [ ]:
# creating more than one named subquery    
       WITH
           [alias_name] AS ([subquery]),
           [alias_name_2] AS ([subquery_2]),
           [alias_name_3] AS ([subquery_3])
       SELECT [main_query]

In [ ]:
# use multiple named subqueries in a WITH clause to gather total sales data on customers 
#   from India:
#   The first named subquery should return all customers that are from India.
#   The second named subquery should calculate the sum total for every customer.
#   The main query should join the two named subqueries, resulting in following final columns:
#       customer_name - The first_name and last_name of the customer, separated by a space
#       total_purchases - The total amount spent on purchases by that customer.
# Results should be sorted by the customer_name column in alphabetical order.
    WITH 
        c_india AS (SELECT * FROM customer WHERE country='India'),
        sum_total AS (SELECT i.customer_id, SUM(i.total) total_purchases
                      FROM invoice i 
                      INNER JOIN customer c ON c.customer_id = i.customer_id
                      GROUP BY i.customer_id
                      )
    SELECT first_name || " " || last_name customer_name,
           total_purchases 
    FROM (SELECT * 
          FROM c_india
          INNER JOIN sum_total ON c_india.customer_id=sum_total.customer_id)
    ORDER BY customer_name ASC

In [ ]:
    WITH 
        c_india AS (SELECT * FROM customer WHERE country='India'),
        sales_per_customer AS (SELECT customer_id, 
                                      SUM(total) total
                               FROM invoice 
                               GROUP BY 1)
    SELECT ci.first_name || " " || ci.last_name customer_name,
           spc.total total_purchases 
    FROM c_india ci
    INNER JOIN sales_per_customer spc ON ci.customer_id = spc.customer_id
    ORDER BY 1

In [ ]:
# find the customer from each country that has spent the most money at the store. 
# Return the following columns, in order:
#   country - The name of each country that we have a customer from.
#   customer_name - The first_name and last_name of the customer from that country 
#                   with the most total purchases.
#   total_purchased - The total dollar amount that customer has purchased.
# The rows should be order by the country column in alphabetical order
    WITH 
        cust_purchase AS (SELECT customer_id, 
                                 SUM(total) total_purchases
                          FROM invoice
                          GROUP BY 1),
        top_cust_per_country AS 
                        (SELECT c.*, 
                                MAX(cp.total_purchases) top_pur 
                         FROM customer c
                         INNER JOIN cust_purchase cp ON c.customer_id = cp.customer_id
                         GROUP BY c.country)
SELECT c.country, 
       c.first_name || " " || c.last_name customer_name,
       tcc.top_pur total_purchased
FROM customer c
INNER JOIN top_cust_per_country tcc ON tcc.customer_id = c.customer_id
ORDER BY 1;